In [11]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from datetime import date
import json
from pprint import pprint
# Import API key
from attom_key import api_key



In [22]:
atx_zip_codes = [78701, 78702, 78703, 78704, 78705, 78751]



headers = {"apikey": api_key, 
           "accept": "application/json"}

params_zip = {"propertytype": "SFR",
              "page": "1",
              
              "pagesize": "1000",
             }

zip_id_url = "https://search.onboard-apis.com/propertyapi/v1.0.0/property/address"

propid_list = []  

for zipcodes in atx_zip_codes:
    params_zip["postalcode"] = zipcodes
    
    zip_results = requests.get(zip_id_url, headers=headers, params=params_zip).json()
    
    results = zip_results["property"]

    for i in range(len(results)):
        propid_zip = {"propid": results[i]["identifier"]["obPropId"], "zip": results[i]["address"]["postal1"]}
        propid_list.append(propid_zip)



In [23]:
atx_zips_df = pd.DataFrame(propid_list)
                                                      
atx_zips_df.head()

,propid,zip
0,14613366148453,78702
1,16678666048453,78702
2,16678667148453,78702
3,16678666148453,78702
4,4759621548453,78702


In [25]:
atx_zips_df["appraisal_2017"] = ""
atx_zips_df["appraisal_2016"] = ""
atx_zips_df["appraisal_2015"] = ""
atx_zips_df["appraisal_2014"] = ""
atx_zips_df["appraisal_2013"] = ""

atx_zips_df.head()

,propid,zip,appraisal_2017,appraisal_2016,appraisal_2015,appraisal_2014,appraisal_2013
0,14613366148453,78702,,,,,
1,16678666048453,78702,,,,,
2,16678667148453,78702,,,,,
3,16678666148453,78702,,,,,
4,4759621548453,78702,,,,,


In [26]:
headers = {"apikey": api_key, 
           "accept": "application/json"}

params_ass = {}

ass_url = "https://search.onboard-apis.com/propertyapi/v1.0.0/assessmenthistory/detail"

# use iterrows to iterate through pandas dataframe
for index, row in atx_zips_df.iterrows():

    # get restaurant type from df
    propid = row["propid"]

    # add keyword to params dict
    params_ass["id"] = propid

    # assemble url and make API request
    ass_results = requests.get(ass_url, headers=headers, params=params_ass).json()
    
    # extract results
    results_ass = ass_results["property"][0]["assessmenthistory"]
    

    
    try:

        
        atx_zips_df.loc[index,"appraisal_2017"] = results_ass[1]["appraised"]["apprttlvalue"]
       
        atx_zips_df.loc[index,"appraisal_2016"] = results_ass[0]["appraised"]["apprttlvalue"]
        
        atx_zips_df.loc[index,"appraisal_2015"] = results_ass[2]["appraised"]["apprttlvalue"]
      
        atx_zips_df.loc[index,"appraisal_2014"] = results_ass[3]["appraised"]["apprttlvalue"]
      
        atx_zips_df.loc[index,"appraisal_2013"] = results_ass[4]["appraised"]["apprttlvalue"]

    
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        


In [27]:
atx_zips_df

,propid,zip,appraisal_2017,appraisal_2016,appraisal_2015,appraisal_2014,appraisal_2013
0,14613366148453,78702,364690,291203,279550,239544,189706
1,16678666048453,78702,620357,721529,534237,490896,387954
2,16678667148453,78702,487057,489671,385432,343528,245135
3,16678666148453,78702,654480,790562,683273,603030,463386
4,4759621548453,78702,336769,268244,255728,227354,162975
5,16602226348453,78702,473405,276086,345169,291544,209168
6,4442096648453,78702,865137,917886,643887,556703,110000
7,16678666248453,78702,364903,321491,312471,266942,163103
8,5080471148453,78702,1047246,1241888,1010125,832768,427969
9,3371319348453,78702,843101,815108,582872,537661,363920


In [28]:
atx_zips_df.replace("", np.nan, inplace=True)
atx_zips_df.dropna(axis=0, inplace=True)
print(atx_zips_df.count())


propid            10
zip               10
appraisal_2017    10
appraisal_2016    10
appraisal_2015    10
appraisal_2014    10
appraisal_2013    10
dtype: int64


In [ ]:
atx_zips_df.to_csv("assessment_history_by_zipcodes.csv")